# Root zone soil moisture (0-100 cm)

In [9]:
#|export
import h5py
import numpy as np
FILE_NAME = '/Users/swapnil/Desktop/API/DATASET/rootzone_soil_moisture/SMAP_L4_SM_gph_20231026T013000_Vv7030_001.h5'
    
f= h5py.File(FILE_NAME,mode='r')

name = '/Geophysical_Data/sm_rootzone'
data = f[name][:]
units = f[name].attrs['units']
units = units.decode('ascii', 'replace')
long_name = f[name].attrs['long_name']
long_name = long_name.decode('ascii', 'replace')
_FillValue = f[name].attrs['_FillValue']
valid_max = f[name].attrs['valid_max']
valid_min = f[name].attrs['valid_min']        
invalid = np.logical_or(data > valid_max,
                            data < valid_min)
invalid = np.logical_or(invalid, data == _FillValue)
data[invalid] = np.nan
#data = np.ma.masked_where(np.isnan(data), data)
    
# Get the geolocation data
latitude = f['/cell_lat'][:]
longitude = f['/cell_lon'][:]

The spatial coordinate Datasets (EASE2_global_projection, latitude, longitude, x coordinates, y coordinates, row index, and
column index) are the same for these five Data Groups and are stored in the HDF5 root Data Group (section 2.2.2). All the
HDF5 Datasets within the five Data Groups have LatCell_LonCell_Array shape. The LatCell_LonCell_Array shape
describes a two-dimensional array, where each array element represents a specific grid cell in the 9 km global cylindrical
EASE-Grid 2.0 as specified by the cell_lat and cell_lon arrays. For example, array element
surface_temp(234,789) in the Geophysical Data Group represents the land surface temperature of the grid cell located
at cell_lat(234,789) and cell_lon(234,789), where cell_row(234,789)=234 and cell_column (234,789)=789.

In [10]:
#long_name

In [11]:
#latitude[234][789]

In [12]:
#longitude[234][789]

This func takes a latitude(or longitude) and returns nearest value avaliable in the dataset, while making sure that we remain in the bounds of the data

In [13]:
def find_nearest_value(array, x , lower_limit, upper_limit):
    
    #currently ignoring that only latitudes from 84.xx to -84.xx available in dataset
    if(x< lower_limit or x> upper_limit):
        raise Exception("invalid ip,out of range") 
    
    # Flatten the 2D latitudes array into a 1D array
    flat_array = array.flatten()

    # Find the index of the nearest latitude in the 1D array
    nearest_index = np.argmin(np.abs(flat_array - x))

    # Get the nearest latitude value
    nearest_value = flat_array[nearest_index]

    return nearest_value

In [14]:
#find_nearest_value(latitude,45,-90,90)

array_name | dtype | lower_limit | upper_limit | unit

lat_array | Float32 | -90.0 | 90.0 | degrees

lon_array | Float32 | -180.0 | 179.999 | degrees

In [15]:
def get_value_for_lat_lon(latitude, longitude, lat_matrix, long_matrix, data):
    # Find the indices where lat_matrix and long_matrix match the provided latitude and longitude
    
    latitude = find_nearest_value(array= lat_matrix,
                                  x=latitude,
                                  lower_limit=-90.0,
                                 upper_limit=90.0)
    
    longitude = find_nearest_value(array= long_matrix,
                                  x=longitude,
                                  lower_limit=-180.0,
                                  upper_limit=179.999)
    
    indices = np.where((lat_matrix == latitude) & (long_matrix == longitude))

    # Check if any matching indices were found
    if indices[0].size > 0:
        # Get the corresponding value from the 'data' array
        value = data[indices[0][0]][indices[1][0]]
        return value
    else:
        # Handle the case when no matching values are found
        return None


# SAMPLE TEST RUN FOR lat, lon = 28.765537, 77.483395 (Muradnagar)

In [16]:
print(get_value_for_lat_lon(28.1,77.483395,latitude,longitude,data))

0.21678248
